In [26]:
import sklearn
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from PyRadioLoc.Utils.GeoUtils import GeoUtils
from sklearn.ensemble import RandomForestRegressor

def write_to_csv(x_pred, y_pred, file_path):
    labels = ["rssi_1_1", "rssi_1_2", "rssi_1_3", "rssi_2_1", "rssi_2_2", "rssi_2_3", "rssi_3_1", "rssi_3_2", "rssi_3_3"]
    y_pred_dataframe = pd.DataFrame(y_pred, columns=labels)
    res = pd.concat([x_pred, y_pred_dataframe], axis=1)
    res.to_csv(path_or_buf=file_path, index = False)

In [13]:
db_train = pd.read_csv("database/LocTreino.csv")
lon_train = db_train["lon"]
lat_train = db_train["lat"]
X_train = pd.concat([lat_train, lon_train], axis=1)
X_train.head()

lat       lon
0 -8.05438 -34.95408
1 -8.05337 -34.94889
2 -8.05390 -34.95955
3 -8.05561 -34.95431
4 -8.05562 -34.95405

In [14]:
y_train = db_train.drop("delay_1", axis = 1)
y_train = y_train.drop("delay_2", axis = 1)
y_train = y_train.drop("delay_3", axis = 1)
y_train = y_train.drop("pontoId", axis = 1)
y_train = y_train.drop("lat", axis = 1)
y_train = y_train.drop("lon", axis = 1)
y_train.head()

rssi_1_1   rssi_1_2   rssi_1_3   rssi_2_1  rssi_2_2   rssi_2_3   rssi_3_1  \
0 -89.044286 -96.667143 -91.320000 -96.106667   -94.480 -90.187143 -94.125000   
1 -91.970000 -89.548571 -88.555714 -82.520000   -80.445 -89.910000 -88.916667   
2 -85.255714 -99.635714 -86.551429 -94.520000  -100.225 -88.232857 -78.955000   
3 -90.025714 -96.200000 -91.204286 -98.160000   -91.935 -92.671429 -78.636667   
4 -94.525714 -96.200000 -93.830000 -98.216667   -94.670 -94.844286 -79.065000   

   rssi_3_2  rssi_3_3  
0   -85.182  -96.4200  
1   -91.602  -97.2550  
2   -92.080  -66.4100  
3   -68.340  -92.7800  
4   -68.250  -96.2125

In [27]:
X_test = pd.read_csv("grid/grid_20.csv")
X_test.head()

lat        lon
0 -8.05955 -34.959850
1 -8.05955 -34.959648
2 -8.05955 -34.959446
3 -8.05955 -34.959244
4 -8.05955 -34.959042

In [32]:
reg_cso = RandomForestRegressor(n_estimators = 115, random_state = 42, criterion = 'mae')
reg_cso.fit(X_train, y_train)
y_pred_cso = reg_cso.predict(X_test)

In [33]:
write_to_csv(X_test, y_pred_cso, "fingerprint_random_forest_grid_20.csv")

In [31]:
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators': list(range(100, 125)), 'criterion': ["mse", "mae"]}
random_forest = RandomForestRegressor()

grid = GridSearchCV(
    random_forest,
    parameters,
    cv = 10,
    scoring = 'neg_mean_squared_error',
    return_train_score = True,
    refit = True
)
grid.fit(X_train, y_train)
grid.best_params_

{'criterion': 'mae', 'n_estimators': 115}